<a href="https://colab.research.google.com/github/CargofirstQaho/Commodity_analysis_GC/blob/main/chickpeawhite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade pip setuptools wheel
!pip install --only-binary=:all: opencv-python-headless==4.8.0.76 numpy==1.24.4 pillow==9.5.0 scikit-image==0.21.0
!pip install streamlit==1.38.0 requests watchdog
!pip install pyngrok==7.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.3 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


ERROR: Could not find a version that satisfies the requirement numpy==1.24.4 (from versions: 1.26.0, 1.26.1, 1.26.2, 1.26.3, 1.26.4, 2.0.0, 2.0.1, 2.0.2, 2.1.0, 2.1.1, 2.1.2, 2.1.3, 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.2.4, 2.2.5, 2.2.6, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.3.5)
ERROR: No matching distribution found for numpy==1.24.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 95.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 116.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 144.3 MB/s  0:00:00
  Attempting uninstall: watchdog
    Found existing installation: watchdog 6.0.0
    Uninstalling watchdog-6.0.0:
      Successfully uninstalled watchdog-6.0.0
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.1.2
    Uninstalling tenacity-9.1.2:
      Successfully uninstalled tenacity-9.1.2
  Attempting uninstall: pillow
    Found existing installation: pillow 11.3.0
    Uninstalling pillow-11.3.0:
      Successfull

In [1]:
from getpass import getpass
import os

ngrok_token = getpass("Enter your ngrok authtoken (leave blank to skip): ")
if ngrok_token:
    os.environ["NGROK_AUTHTOKEN"] = ngrok_token.strip()
    print("✅ ngrok token set.")
else:
    print("⚠️ ngrok token not provided, will use Cloudflare tunnel instead.")


Enter your ngrok authtoken (leave blank to skip): ··········
⚠️ ngrok token not provided, will use Cloudflare tunnel instead.


In [2]:
app_code = r"""
import streamlit as st
import numpy as np
import cv2
from PIL import Image, UnidentifiedImageError
from io import BytesIO
import math

# --- Page Setup ---
st.set_page_config(page_title='Chickpea Analyzer', layout='wide')
st.title('Chickpea Quality Analyzer')

st.markdown('''
Upload JPG/PNG or capture via camera.
Automatically detects chickpeas, measures size, and classifies quality.

''')

# --- Fixed Thresholds for Production ---
BROKEN_RATIO = 0.7
DISCOLORED_PERCENTILE = 20
SHRIVELED_PERCENTILE = 75
MIN_AREA = 200
FOREIGN_SOLIDITY = 0.5
FOREIGN_AREA_MULT = 5.0
FOREIGN_ASPECT_RATIO = 3.0
MARKER_SIZE_MM = 8.0

# --- Image Processing Functions ---
def enhance_contrast(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    l = clahe.apply(l)
    lab = cv2.merge((l,a,b))
    return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

def preprocess(img):
    img_enh = enhance_contrast(img)
    gray = cv2.cvtColor(img_enh, cv2.COLOR_BGR2GRAY)
    blur = cv2.bilateralFilter(gray, 9, 75, 75)
    th = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY_INV, 51, 5)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    opened = cv2.morphologyEx(th, cv2.MORPH_OPEN, kernel, iterations=2)
    dist = cv2.distanceTransform(opened, cv2.DIST_L2, 5)
    _, fg = cv2.threshold(dist, 0.35*dist.max(), 255, 0)
    fg = np.uint8(fg)
    unknown = cv2.subtract(opened, fg)
    _, markers = cv2.connectedComponents(fg)
    markers = markers + 1
    markers[unknown==255] = 0
    markers = cv2.watershed(cv2.cvtColor(img_enh, cv2.COLOR_BGR2RGB), markers)
    mask = np.zeros_like(gray, np.uint8)
    mask[markers>1] = 255
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    return img_enh, gray, mask

def extract_contours(mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return [c for c in contours if cv2.contourArea(c) >= MIN_AREA]

def grain_features(gray, contour):
    area = cv2.contourArea(contour)
    rect = cv2.minAreaRect(contour)
    (cx, cy), (w, h), _ = rect
    length = max(w, h); width = min(w,h)
    aspect_ratio = (length+1)/(width+1)
    mask = np.zeros_like(gray)
    cv2.drawContours(mask, [contour], -1, 255, -1)
    mean_intensity = cv2.mean(gray, mask=mask)[0]
    hull = cv2.convexHull(contour)
    solidity = area / max(cv2.contourArea(hull), 1.0)
    circularity = (4*math.pi*area)/(cv2.arcLength(contour, True)**2+1e-6)
    return {"contour": contour, "area": area, "length_px": length, "width_px": width,
            "aspect_ratio": aspect_ratio, "mean_intensity": mean_intensity,
            "solidity": solidity, "circularity": circularity, "cx": cx, "cy": cy}

def classify_grains(feats):
    if not feats: return [], {}
    lengths = np.array([f["length_px"] for f in feats])
    intensities = np.array([f["mean_intensity"] for f in feats])
    areas = np.array([f["area"] for f in feats])
    median_len = np.median(lengths)
    broken_thresh = BROKEN_RATIO * median_len
    shriveled_thresh = np.percentile(intensities, SHRIVELED_PERCENTILE)
    discolored_thresh = np.percentile(intensities, DISCOLORED_PERCENTILE)
    median_area = np.median(areas)
    results = []
    for f in feats:
        broken = f["length_px"] < broken_thresh
        shriveled = f["mean_intensity"] >= shriveled_thresh
        discolored = f["mean_intensity"] <= discolored_thresh
        foreign = (f["solidity"] < FOREIGN_SOLIDITY or f["area"] > FOREIGN_AREA_MULT*median_area
                   or f["aspect_ratio"] > FOREIGN_ASPECT_RATIO)
        if foreign: label = "foreign"
        elif broken: label = "broken"
        elif discolored: label = "discolored"
        elif shriveled: label = "shriveled"
        else: label = "normal"
        results.append({**f, "label": label})
    meta = {
        "median_len_px": float(median_len),
        "broken_threshold_px": float(broken_thresh),
        "shriveled_threshold": float(shriveled_thresh),
        "discolored_threshold": float(discolored_thresh),
        "median_area_px": float(median_area)
    }
    return results, meta

def annotate(img, results):
    annotated = img.copy()
    colors = {"normal": (0,255,0), "broken": (0,0,255),
              "shriveled": (0,165,255), "discolored": (128,0,128),
              "foreign": (255,0,0)}
    for r in results:
        c = r["contour"]; label = r["label"]; col = colors[label]
        cv2.drawContours(annotated, [c], -1, col, 2)
        cx, cy = int(r["cx"]), int(r["cy"])
        cv2.putText(annotated, label, (cx+5, cy-5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, col, 1)
    return annotated

def pixels_to_mm(px): return MARKER_SIZE_MM/px if px>0 else 0

# --- Main App ---
uploaded = st.file_uploader("Upload image", type=["jpg","jpeg","png"])
camera = st.camera_input("Or capture using camera")
image_bytes = uploaded.read() if uploaded else camera.getvalue() if camera else None
if not image_bytes:
    st.info("Upload or capture an image to start.")
    st.stop()

try:
    pil = Image.open(BytesIO(image_bytes)).convert("RGB")
except UnidentifiedImageError:
    st.error("Invalid image format"); st.stop()

img = np.array(pil)[:,:,::-1]
img_enh, gray, mask = preprocess(img)
contours = extract_contours(mask)
feats = [grain_features(gray,c) for c in contours]
results, meta = classify_grains(feats)

# --- Counts & Metrics ---
total = len(results)
count = {k: sum(r["label"]==k for r in results) for k in ["normal","broken","shriveled","discolored","foreign"]}
avg_len_px = np.mean([r["length_px"] for r in results]) if total else 0
avg_wid_px = np.mean([r["width_px"] for r in results]) if total else 0
mm_per_px = pixels_to_mm(avg_len_px)
avg_len_mm, avg_wid_mm = avg_len_px*mm_per_px, avg_wid_px*mm_per_px

# --- Display Results ---
col1,col2 = st.columns(2)
with col1:
    st.subheader("Counts")
    st.metric("Total", total)
    for k in count: st.metric(k.capitalize(), count[k])
    st.subheader("Average Size (mm)")
    st.write(f"Length: **{avg_len_mm:.2f} mm**")
    st.write(f"Width: **{avg_wid_mm:.2f} mm**")
    st.caption("Size calibration uses default 8mm chickpea size.")
with col2:
    st.subheader("Annotated Image")
    annotated = annotate(img, results)
    st.image(annotated[:,:,::-1], use_column_width=True)
    buf = BytesIO(); Image.fromarray(annotated[:,:,::-1]).save(buf,"PNG")
    st.download_button("Download Annotated Image", buf.getvalue(),
                       file_name="annotated.png", mime="image/png")


"""

with open("app.py","w") as f:
    f.write(app_code)
print("✅ app.py written successfully with accuracy improvements.")


✅ app.py written successfully with accuracy improvements.


In [3]:
!wget -q -O cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared

In [ ]:
# Assuming you already ran the previous cell that writes app.py
# Just ensure this is present:
!ls app.py


app.py


In [ ]:
import subprocess
import time

# Start Streamlit headlessly
streamlit_proc = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

print("🚀 Streamlit started on localhost:8501…")
time.sleep(5)  # Give it a moment to boot


🚀 Streamlit started on localhost:8501…


In [ ]:
import subprocess, shlex, re, time

print("🌐 Starting Cloudflare Tunnel…")
cmd = "./cloudflared tunnel --url http://localhost:8501 --no-autoupdate"
cloud = subprocess.Popen(shlex.split(cmd), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

public_url = None
start = time.time()

while True:
    line = cloud.stdout.readline().strip()
    if line:
        print(line)
        m = re.search(r"https://[a-zA-Z0-9.-]+trycloudflare.com", line)
        if m:
            public_url = m.group(0)
            break

    if time.time() - start > 25:
        print("❌ Cloudflare took too long, try re-running.")
        break

if public_url:
    print("\n✅ PUBLIC URL:", public_url)
else:
    print("❌ Failed to get public URL, check Cloudflare logs.")


🌐 Starting Cloudflare Tunnel…
2025-11-24T05:30:51Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-24T05:30:51Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-11-24T05:30:56Z INF +--------------------------------------------------------------------------------------------+
2025-11-24T05:30:56Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-11-24T05:30:56Z INF |  https://button-newbie-c